# Entendendo e explorando os dados

A base de dados foi coletada do [kaggle](https://www.kaggle.com/datasets/kanchana1990/perfume-e-commerce-dataset-2024?select=ebay_womens_perfume.csv) com cerca de 2mil itens de vendas de perfumes femininos e masculinos em um -ecommerce

### 0. Imports

In [18]:
import pandas as pd
import re


## 1. Base de dados

In [19]:
df1 = pd.read_csv('C:\\Users\\Cliente\\Documents\\Git\\case\\docs\\ebay_womens_perfume.csv')
df2 = pd.read_csv('C:\\Users\\Cliente\\Documents\\Git\\case\\docs\\ebay_mens_perfume.csv')

df = pd.concat([df1, df2])
df.head()

,brand,title,type,price,priceWithCurrency,available,availableText,sold,lastUpdated,itemLocation
0,Carolina Herrera,Good Girl by Carolina Herrera 2.7 oz Eau De Pa...,Eau de Parfum,43.99,US $43.99/ea,2.0,2 available / 393 sold,393.0,"May 23, 2024 10:43:50 PDT","Thomasville, Alabama, United States"
1,As Shown,Parfums de Marly Delina La Rosee Eau de Parfum...,Eau de Parfum,79.99,US $79.99,5.0,5 available / 40 sold,40.0,"May 24, 2024 00:15:48 PDT","New Jersey, Hong Kong"
2,PRADA,PRADA Paradoxe by Prada EDP 3.0oz/90ml Spray P...,Eau de Parfum,59.99,US $59.99,10.0,More than 10 available / 35 sold,35.0,"May 14, 2024 20:54:25 PDT","Orange, New Jersey, United States"
3,As Show,J'adore Parfum D'eau by Christian 3.4 oz EDP F...,Eau de Parfum,59.99,US $59.99/ea,10.0,More than 10 available / 9 sold,9.0,"May 23, 2024 01:23:05 PDT","USA, New Jersey, Hong Kong"
4,Khadlaj,Shiyaaka for Men EDP Spray 100ML (3.4 FL.OZ) B...,Eau de Parfum,29.99,US $29.99/ea,10.0,More than 10 available,NaN,NaN,"Little Ferry, New Jersey, United States"


#### Colunas
- brand: A marca do perfume.
- title: O título da listagem.
- type: O tipo de perfume (ex.: Eau de Parfum, Eau de Toilette).
- price: O preço do perfume.
- priceWithCurrency: O preço com a notação da moeda.
- available: O número de itens disponíveis.
- availableText: Descrição textual da disponibilidade.
- sold: O número de itens vendidos.
- lastUpdated: A última atualização do anúncio.
- itemLocation: A localização do item.

### 1.1. Resumo

In [20]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
Index: 2000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   brand              1998 non-null   object 
 1   title              2000 non-null   object 
 2   type               1995 non-null   object 
 3   price              2000 non-null   float64
 4   priceWithCurrency  2000 non-null   object 
 5   available          1758 non-null   float64
 6   availableText      1989 non-null   object 
 7   sold               1978 non-null   float64
 8   lastUpdated        1874 non-null   object 
 9   itemLocation       2000 non-null   object 
dtypes: float64(3), object(7)
memory usage: 171.9+ KB


,price,available,sold
count,2000.000000,1758.000000,1978.000000
mean,43.187090,20.728669,632.473711
std,32.619625,56.781389,2470.055822
min,1.990000,2.000000,1.000000
25%,21.972500,5.000000,14.000000
50%,34.040000,10.000000,51.000000
75%,53.990000,10.000000,285.750000
max,299.990000,842.000000,54052.000000


- Existem alguns valores nulos que devemos analisar
- existem alguns outliers em "sold"

In [21]:
brands_count = df['brand'].nunique()
type_count = df['type'].nunique()
location_count =  df['itemLocation'].nunique()

print(f'{brands_count} marcas unicas, {type_count} tipos de perfume unicos e {location_count} localidades unicas')

401 marcas unicas, 116 tipos de perfume unicos e 447 localidades unicas


## 2. Limpeza

### 2.1. Moeda de venda

In [22]:
# validando se todos os preços são em dolar americano mesmo

us_count = df['priceWithCurrency'].str.contains('US').sum()
print(f"{us_count} linhas são dolares americano")

1997 linhas são dolares americano


In [23]:
df[~df['priceWithCurrency'].str.contains('US')]

,brand,title,type,price,priceWithCurrency,available,availableText,sold,lastUpdated,itemLocation
352,Lucianno,Lucianno California Vibe M 100ml Boxed,Extrait De Parfum,66.0,C $66.00,10.0,More than 10 available / 21 sold,21.0,"May 24, 2024 02:33:19 PDT","Scarborough, Ontario, Canada"
379,Al Wataniah,Al Wataniah Sabah Al Ward EDP M 100ml Boxed,Eau De Parfum,50.0,C $50.00,10.0,More than 10 available / 19 sold,19.0,"May 23, 2024 16:03:42 PDT","Scarborough, Ontario, Canada"
957,David Beckham,David Beckham The Essence Man 75ml Boxed,Eau de Toilette,37.0,C $37.00,10.0,More than 10 available / 60 sold,60.0,"May 21, 2024 07:32:36 PDT","Scarborough, Ontario, Canada"


In [24]:
# 3 itens são dolares canadenses, vamos corrigir isso

def convert_canadian_to_usd(price):
    return price * 0.69

def adjusting_price_to_usd(df):
    df.loc[df['priceWithCurrency'].str.contains('C'), 'price'] = df.loc[df['priceWithCurrency'].str.contains('C'), 'price'].apply(convert_canadian_to_usd)
    
    df['price'] = df['price'].round(2)
    return df


df = adjusting_price_to_usd(df)

### 2.2. Eliminando colunas desnecessárias

In [25]:
## da pra perceber que as colunas availableText e priceWithCurrency são inurteis, entao vamos remove-las
df = df.drop(columns=['availableText', 'priceWithCurrency'], axis=1)
df

,brand,title,type,price,available,sold,lastUpdated,itemLocation
0,Carolina Herrera,Good Girl by Carolina Herrera 2.7 oz Eau De Pa...,Eau de Parfum,43.99,2.0,393.0,"May 23, 2024 10:43:50 PDT","Thomasville, Alabama, United States"
1,As Shown,Parfums de Marly Delina La Rosee Eau de Parfum...,Eau de Parfum,79.99,5.0,40.0,"May 24, 2024 00:15:48 PDT","New Jersey, Hong Kong"
2,PRADA,PRADA Paradoxe by Prada EDP 3.0oz/90ml Spray P...,Eau de Parfum,59.99,10.0,35.0,"May 14, 2024 20:54:25 PDT","Orange, New Jersey, United States"
3,As Show,J'adore Parfum D'eau by Christian 3.4 oz EDP F...,Eau de Parfum,59.99,10.0,9.0,"May 23, 2024 01:23:05 PDT","USA, New Jersey, Hong Kong"
4,Khadlaj,Shiyaaka for Men EDP Spray 100ML (3.4 FL.OZ) B...,Eau de Parfum,29.99,10.0,NaN,NaN,"Little Ferry, New Jersey, United States"
...,...,...,...,...,...,...,...,...
995,GUESS,Guess 1981 by Guess cologne for men EDT 3.3 / ...,Eau de Toilette,20.28,45.0,1613.0,"May 24, 2024 08:14:07 PDT","Dallas, Texas, United States"
996,Armaf,Club de Nuit Intense by Armaf cologne for men ...,Eau de Toilette,30.58,10.0,31.0,"May 23, 2024 08:39:30 PDT",United States
997,Paco Rabanne,Invictus by Paco Rabanne for Men EDT Spray 3.4...,Eau de Toilette,39.99,2.0,305.0,"May 23, 2024 15:27:18 PDT","Jamaica, New York, United States"
998,Lomani,"Lomani EDT Cologne 3.4 oz Men - Authentic, Bra...",Eau de Toilette,9.99,2.0,22.0,"May 20, 2024 13:20:54 PDT","Lincoln Park, Michigan, United States"


## 3. Feature Engineering

In [26]:
df['total_sale'] = (df['price'] * df['sold']).round(2)
df.head()

,brand,title,type,price,available,sold,lastUpdated,itemLocation,total_sale
0,Carolina Herrera,Good Girl by Carolina Herrera 2.7 oz Eau De Pa...,Eau de Parfum,43.99,2.0,393.0,"May 23, 2024 10:43:50 PDT","Thomasville, Alabama, United States",17288.07
1,As Shown,Parfums de Marly Delina La Rosee Eau de Parfum...,Eau de Parfum,79.99,5.0,40.0,"May 24, 2024 00:15:48 PDT","New Jersey, Hong Kong",3199.60
2,PRADA,PRADA Paradoxe by Prada EDP 3.0oz/90ml Spray P...,Eau de Parfum,59.99,10.0,35.0,"May 14, 2024 20:54:25 PDT","Orange, New Jersey, United States",2099.65
3,As Show,J'adore Parfum D'eau by Christian 3.4 oz EDP F...,Eau de Parfum,59.99,10.0,9.0,"May 23, 2024 01:23:05 PDT","USA, New Jersey, Hong Kong",539.91
4,Khadlaj,Shiyaaka for Men EDP Spray 100ML (3.4 FL.OZ) B...,Eau de Parfum,29.99,10.0,NaN,NaN,"Little Ferry, New Jersey, United States",NaN


### 3.1. Localização

In [27]:
df['country'] = df['itemLocation'].str.split(', ').str[-1]
df['state'] = df['itemLocation'].str.split(', ').str[-2]
df['city'] = df['itemLocation'].str.split(', ').str[0]
df = df.drop(columns=['itemLocation'], axis=1)
df.head()

,brand,title,type,price,available,sold,lastUpdated,total_sale,country,state,city
0,Carolina Herrera,Good Girl by Carolina Herrera 2.7 oz Eau De Pa...,Eau de Parfum,43.99,2.0,393.0,"May 23, 2024 10:43:50 PDT",17288.07,United States,Alabama,Thomasville
1,As Shown,Parfums de Marly Delina La Rosee Eau de Parfum...,Eau de Parfum,79.99,5.0,40.0,"May 24, 2024 00:15:48 PDT",3199.60,Hong Kong,New Jersey,New Jersey
2,PRADA,PRADA Paradoxe by Prada EDP 3.0oz/90ml Spray P...,Eau de Parfum,59.99,10.0,35.0,"May 14, 2024 20:54:25 PDT",2099.65,United States,New Jersey,Orange
3,As Show,J'adore Parfum D'eau by Christian 3.4 oz EDP F...,Eau de Parfum,59.99,10.0,9.0,"May 23, 2024 01:23:05 PDT",539.91,Hong Kong,New Jersey,USA
4,Khadlaj,Shiyaaka for Men EDP Spray 100ML (3.4 FL.OZ) B...,Eau de Parfum,29.99,10.0,NaN,NaN,NaN,United States,New Jersey,Little Ferry


In [28]:
## removendo os outliers
# mais de 5mil produtos vendidos é um outlier assim como 0 prodtos vendidos é irrelevante

# outlier = df[df['sold'] > 3000]
# df = df[df['sold'] <= 3000]
# df = df[df['sold'] != 0]

### 3.2. Nulos

In [29]:
null_counts = df.isnull().sum()
print(null_counts)

brand            2
title            0
type             5
price            0
available      242
sold            22
lastUpdated    126
total_sale      22
country          0
state            3
city             0
dtype: int64


In [30]:
# sabendoq ue as linhas nulas de data são 7% da base, vamos eliminá-las
#vamos eliminálas poisé menos de 10% da base e seria muito arriscado supor quais perfumes foram vendidos nestas datas inexistentes

df = df.dropna(subset=['lastUpdated'])

In [31]:
df['brand'] = df['brand'].fillna('Unknown')
df['state'] = df['state'].fillna('Unknown')
df['type'] = df['type'].fillna('Eau de Parfum')
df['available'] = df['available'].fillna(int(0))
df['sold'] = df['sold'].fillna(int(0))

df['sold'] = df['sold'].astype(int)
df['available'] = df['available'].astype(int)

### 3.3. Datas

In [32]:
def extract_date_parts(date_str):
    month = date_str[:3] 
    year = date_str[7:12]
    day_match = re.search(r'(\d{1,2}),', date_str) 
    day = day_match.group(1) if day_match else None
    return pd.Series([month, day, year])

def get_weeknames(df):
    df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
    df['date'] = pd.to_datetime(df['year'] + '-' + df['month'] + '-' + df['day'], format='%Y-%b-%d')
    df['weekday'] = df['date'].dt.day_name()
    df[['month', 'day', 'year']] = df[['month', 'day', 'year']].apply(lambda col: col.str.strip())

    df = df.drop(columns=['date'], axis=1)
    return df

def get_weekend(df):
    df['is_weekend'] = df['weekday'].isin(['Saturday', 'Sunday']).astype(int)
    return df

df = df.dropna(subset=['lastUpdated'])

df[['month', 'day', 'year']] = df['lastUpdated'].apply(extract_date_parts)

# Removendo espaços vazios nas colunas 'month', 'day' e 'year'
df['month'] = df['month'].str.strip()
df['day'] = df['day'].str.strip()
df['year'] = df['year'].str.strip()


month_mapping = {
    'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04', 'May': '05', 'Jun': '06',
    'Jul': '07', 'Aug': '08', 'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'
}

df['day'] = df['day'].fillna(1)
df['day'] = df['day'].astype(int)
df['year'] = df['year'].str.strip().astype(int)
df['month'] = df['month'].map(month_mapping)

df['date'] = pd.to_datetime(
    df['year'].astype(str) + '-' + df['month'].astype(str) + '-' + df['day'].astype(str), 
    format='%Y-%m-%d',
    errors='coerce'
)

df['weekday'] = df['date'].dt.day_name()
df = get_weekend(df)
df = df.drop(columns=['lastUpdated'], axis=1)
df.head()

,brand,title,type,price,available,sold,total_sale,country,state,city,month,day,year,date,weekday,is_weekend
0,Carolina Herrera,Good Girl by Carolina Herrera 2.7 oz Eau De Pa...,Eau de Parfum,43.99,2,393,17288.07,United States,Alabama,Thomasville,05,23,2024,2024-05-23,Thursday,0
1,As Shown,Parfums de Marly Delina La Rosee Eau de Parfum...,Eau de Parfum,79.99,5,40,3199.60,Hong Kong,New Jersey,New Jersey,05,24,2024,2024-05-24,Friday,0
2,PRADA,PRADA Paradoxe by Prada EDP 3.0oz/90ml Spray P...,Eau de Parfum,59.99,10,35,2099.65,United States,New Jersey,Orange,05,14,2024,2024-05-14,Tuesday,0
3,As Show,J'adore Parfum D'eau by Christian 3.4 oz EDP F...,Eau de Parfum,59.99,10,9,539.91,Hong Kong,New Jersey,USA,05,23,2024,2024-05-23,Thursday,0
5,Viktor & Rolf,Flowerbomb by Viktor & Rolf 3.4 oz 100ML Eau D...,Eau de Parfum,51.99,8,184,9566.16,United States,Michigan,Warren,05,23,2024,2024-05-23,Thursday,0


In [33]:
df.to_csv('C:\\Users\\Cliente\\Documents\\Git\\case\\docs\\parfum_sales.csv', index=False)